In [1]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, CategoricalArrays
using Plots
using Distributions

In [2]:
const GRB_ENV = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-11


In [3]:
RHO = -0.5;

In [4]:
df = CSV.read("imputed_table.csv", DataFrame)[:, 2:end];

people = CSV.read("people.csv", DataFrame);

In [5]:
people_df = people;

In [6]:
preference_df = df[1:20, 1:20]

,1.0,2.0,3.0,4.0,5.0,6.0,7.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.12573,-0.132105,0.640423,0.1049,-0.535669,0.361595,1.304
2,-0.504199,0.357955,0.414866,-1.24978,0.175539,-0.320363,-1.90409
3,0.937358,0.21877,0.339794,1.39214,0.31772,0.545022,0.99023
4,0.135331,0.261963,0.7203,-0.553811,-1.29184,-1.6005,0.553141
5,-1.23944,0.00437185,0.335725,1.19403,2.65828,-0.396796,-0.877103
6,-0.613547,1.13328,-0.784445,0.119175,-0.284907,0.00117764,-0.559545
7,0.450715,-0.226007,0.66004,-1.7661,-0.100062,0.861031,0.479559
8,-0.489866,0.541139,-0.336873,0.770053,0.983356,1.98303,-0.412383
9,-0.234397,1.84195,0.398938,-1.88541,-1.23653,1.51647,-0.0490567


In [7]:
m1 = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Create variables
@variable(m1, z[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)
@variable(m1, t[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)

# Create objective
@objective(m1, Min, -1 * sum(z[i,j] * (preference_df[i,j] + preference_df[j,i]) for i in 1:size(preference_df)[1], j in i:size(preference_df)[2]) + (size(preference_df)[1] - sum(z)) * -RHO)

# Create constraints

# Each person can only be matched with one person
@constraint(m1, [i=1:size(preference_df, 1)], sum(z[i,j] for j in 1:size(preference_df, 2)) <= 1)
@constraint(m1, [j=1:size(preference_df, 2)], sum(z[i,j] for i in 1:size(preference_df, 1)) <= 1)

# Make sure that if i is matched with j, then j is matched with i
# Different gender matching
# People with strong religious preferences are matched with someone sharing the same religious preference
religion_df = Matrix(people_df[:, [:agnostic, :atheist, :buddhist, :catholic, :hindu, :jewish, :mormon, :muslim, :protestant, :unaffiliated]])

for i in 1:size(z, 1)
    for j in i:size(z, 2)
        @constraint(m1, z[i,j] == z[j,i])
        #@constraint(m1, (people_df[i, :gender] + people_df[j, :gender] - 1) * z[i,j] ==0 )

        #@constraint(m1, (1 - sum(religion_df[i, k] * religion_df[j, k] for k in 1:size(religion_df)[2])) * t[i,j] * z[i,j] <= 0)
        #@constraint(m1, t[i,j] >= people_df[i, :imprelig])
        #@constraint(m1, t[i,j] >= people_df[j, :imprelig])
    end
end

optimize!(m1)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 250 rows, 800 columns and 1180 nonzeros
Model fingerprint: 0xf3b7ed8a
Variable types: 0 continuous, 800 integer (800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-03, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -16.2809448
Presolve removed 233 rows and 751 columns
Presolve time: 0.02s
Presolved: 17 rows, 49 columns, 98 nonzeros
Found heuristic solution: objective -26.8279622
Variable types: 0 continuous, 49 integer (49 binary)
Found heuristic solution: objective -27.4746954

Root relaxation: objective -2.886995e+01, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -28

In [14]:
write_to_file(m1, "model_orig.lp")

In [12]:
MOI.get(m1, Gurobi.ModelAttribute("IsMIP"))

In [15]:
preference_df

,1.0,2.0,3.0,4.0,5.0,6.0,7.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.12573,-0.132105,0.640423,0.1049,-0.535669,0.361595,1.304
2,-0.504199,0.357955,0.414866,-1.24978,0.175539,-0.320363,-1.90409
3,0.937358,0.21877,0.339794,1.39214,0.31772,0.545022,0.99023
4,0.135331,0.261963,0.7203,-0.553811,-1.29184,-1.6005,0.553141
5,-1.23944,0.00437185,0.335725,1.19403,2.65828,-0.396796,-0.877103
6,-0.613547,1.13328,-0.784445,0.119175,-0.284907,0.00117764,-0.559545
7,0.450715,-0.226007,0.66004,-1.7661,-0.100062,0.861031,0.479559
8,-0.489866,0.541139,-0.336873,0.770053,0.983356,1.98303,-0.412383
9,-0.234397,1.84195,0.398938,-1.88541,-1.23653,1.51647,-0.0490567


In [41]:
value.(z)

5×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0

In [8]:
zij = value.(z);

sum(zij, dims=1)

1×20 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  …  1.0  1.0  1.0  1.0  0.0  1.0  1.0

In [9]:
CSV.write("z.csv", Tables.table(zij))

"z.csv"

## Allow each person to pair with 2 people, no enforced closed groups

In [30]:
preference_df = Matrix(df[1:20, 1:20]);
preference_df[diagind(preference_df)] .= -RHO;

people_df = people;

In [18]:
m2 = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Create variables
@variable(m2, z[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)
@variable(m2, t[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)

# Create objective
@objective(m2, Min, -1 * sum(z[i,j] * (preference_df[i,j] + preference_df[j,i]) for i in 1:size(preference_df, 1), j in i:size(preference_df, 1)) + (size(preference_df, 1) - sum(z)) * RHO)

# Create constraints

# Each person can only be matched with two people
@constraint(m2, [i=1:size(preference_df, 1)], sum(z[i,j] for j in i:size(preference_df, 1)) <= 2)
@constraint(m2, [j=1:size(preference_df, 1)], sum(z[i,j] for i in 1:size(preference_df, 1)) <= 2)

#@constraint(m1, [k=1:size(preference_df, 1)], sum(z[i,j,k] for i in 1:size(preference_df, 1) for j in 1:size(preference_df, 1)) <= 2)

#religion_df = Matrix(people_df[:, [:agnostic, :atheist, :buddhist, :catholic, :hindu, :jewish, :mormon, :muslim, :protestant, :unaffiliated]])

for i in 1:size(z, 1)
    for j in i:size(z, 1)

        # Make sure that if i is matched with j, then j is matched with i
        @constraint(m2, z[i,j] == z[j, i])

        # Different gender matching
        @constraint(m2, (people_df[i, :gender] + people_df[j, :gender] - 1) * z[i,j] ==0 ) 

        # People with strong religious preferences are matched with someone sharing the same religious preference
        # @constraint(m1, (1 - sum(religion_df[i, k] * religion_df[j, k] for k in 1:size(religion_df)[2])) * t[i,j] * z[i,j] <= 0)
        # @constraint(m1, t[i,j] >= people_df[i, :imprelig])
        # @constraint(m1, t[i,j] >= people_df[j, :imprelig])
    end
end

# for i in 1:size(z, 1)
#     for j in i:size(z, 2)
#         @constraint(m1, z[j,i,j] ==0)
#         @constraint(m1, z[j,j,i] ==0)
#         for k in j:size(z, 3)
#             @constraint(m1, z[i,k,j] ==0)
#             @constraint(m1, z[j,k,i] ==0)
#             @constraint(m1, z[k,i,j] ==0)
#             @constraint(m1, z[k,j,i] ==0)
#             @constraint(m1, z[j,i,k] ==0)
#         end
#     end
# end

optimize!(m2)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 460 rows, 800 columns and 1100 nonzeros
Model fingerprint: 0x7203e345
Variable types: 0 continuous, 800 integer (800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -10.0000000
Presolve removed 452 rows and 780 columns
Presolve time: 0.00s
Presolved: 8 rows, 20 columns, 32 nonzeros
Found heuristic solution: objective -23.6744312
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective -24.7943220

Root relaxation: cutoff, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       -

In [ ]:
zij = value.(z);

In [152]:
sum(zij)

15.0

In [23]:
CSV.write("z2.csv", Tables.table(zij))

"z2.csv"

### Requiring closed groups.

In [143]:
# preference_df = Matrix(df[1:10, 1:10]);
# preference_df[diagind(preference_df)] .= -RHO;

# people_df = people;

In [115]:
# m2 = Model(() -> Gurobi.Optimizer(GRB_ENV))

# # Create variables
# @variable(m2, z[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)
# @variable(m2, t[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)

# # Create objective
# @objective(m2, Min, -1 * sum(z[i,j] * (preference_df[i,j] + preference_df[j,i]) for i in 1:size(preference_df, 1), j in i:size(preference_df, 1)) )#+ (size(preference_df, 1) - sum(z)) * -RHO)

# # Create constraints

# # Each person can only be matched with two people
# @constraint(m2, [i=1:size(preference_df, 1)], sum(z[i,j] for j in i:size(preference_df, 1)) <= 2)
# @constraint(m2, [j=1:size(preference_df, 1)], sum(z[i,j] for i in 1:size(preference_df, 1)) <= 2)


# #@constraint(m1, [k=1:size(preference_df, 1)], sum(z[i,j,k] for i in 1:size(preference_df, 1) for j in 1:size(preference_df, 1)) <= 2)

# #religion_df = Matrix(people_df[:, [:agnostic, :atheist, :buddhist, :catholic, :hindu, :jewish, :mormon, :muslim, :protestant, :unaffiliated]])

# for i in 1:size(z, 1)
#     for j in i:size(z, 1)

#         # Make sure that if i is matched with j, then j is matched with i
#         @constraint(m2, z[i,j] == z[j, i])

#         # Different gender matching
#         #@constraint(m2, (people_df[i, :gender] + people_df[j, :gender] - 1) * z[i,j] ==0 )
        

#         # People with strong religious preferences are matched with someone sharing the same religious preference
#         # @constraint(m1, (1 - sum(religion_df[i, k] * religion_df[j, k] for k in 1:size(religion_df)[2])) * t[i,j] * z[i,j] <= 0)
#         # @constraint(m1, t[i,j] >= people_df[i, :imprelig])
#         # @constraint(m1, t[i,j] >= people_df[j, :imprelig])
#     end
#     @constraint(m2, z[i,i] ==0)
# end

# # for i in 1:size(z, 1)
# #     for j in i:size(z, 2)
# #         @constraint(m1, z[j,i,j] ==0)
# #         @constraint(m1, z[j,j,i] ==0)
# #         for k in j:size(z, 3)
# #             @constraint(m1, z[i,k,j] ==0)
# #             @constraint(m1, z[j,k,i] ==0)
# #             @constraint(m1, z[k,i,j] ==0)
# #             @constraint(m1, z[k,j,i] ==0)
# #             @constraint(m1, z[j,i,k] ==0)
# #         end
# #     end
# # end

# optimize!(m2)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 85 rows, 200 columns and 255 nonzeros
Model fingerprint: 0xaaebc4de
Variable types: 0 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 0.0000000
Presolve removed 75 rows and 174 columns
Presolve time: 0.00s
Presolved: 10 rows, 26 columns, 52 nonzeros
Variable types: 0 continuous, 26 integer (26 binary)
Found heuristic solution: objective -15.9252976

Root relaxation: objective -1.768192e+01, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     -17.6819172  -17.68192  0.00%     -   

In [116]:
zij = value.(z)

10×10 Matrix{Float64}:
  0.0   0.0  -0.0   1.0   0.0   0.0   1.0  -0.0   0.0   0.0
  0.0   0.0  -0.0   0.0   0.0   1.0   0.0   0.0   1.0   0.0
 -0.0  -0.0   0.0   1.0  -0.0   0.0   1.0  -0.0  -0.0   0.0
  1.0   0.0   1.0   0.0   0.0   0.0   0.0  -0.0   0.0   0.0
  0.0   0.0  -0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0
  0.0   1.0   0.0   0.0   0.0   0.0  -0.0   1.0  -0.0   0.0
  1.0   0.0   1.0   0.0   0.0  -0.0   0.0   0.0  -0.0  -0.0
 -0.0   0.0  -0.0  -0.0   1.0   1.0   0.0   0.0  -0.0   0.0
  0.0   1.0  -0.0   0.0   0.0  -0.0  -0.0  -0.0   0.0   1.0
  0.0   0.0   0.0   0.0   1.0   0.0  -0.0   0.0   1.0   0.0

In [57]:
RHO=1
preference_df = Matrix(df);
preference_df[diagind(preference_df)] .= -RHO;

people_df = people;

In [58]:
m3 = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Create variables
@variable(m3, z[1:size(preference_df, 1), 1:size(preference_df, 1), 1:size(preference_df, 1)], Bin)
@variable(m3, t[1:size(preference_df, 1), 1:size(preference_df, 1), 1:size(preference_df, 1)], Bin)

# Create objective
#@objective(m3, Min, -1 * sum(z[i,j,k] * (preference_df[i,j] + preference_df[j,i] + preference_df[j,k] + preference_df[k,j] + preference_df[i,k] + preference_df[k,i]) for i in 1:size(preference_df, 1), j in 1:size(preference_df, 1), k in 1:size(preference_df, 1)) )
@objective(m3, Min, -1 * sum(z[i,j,k] * (preference_df[i,j] + preference_df[j,i] + preference_df[j,k] + preference_df[k,j] + preference_df[i,k] + preference_df[k,i]) for i in 1:size(preference_df, 1), j in i:size(preference_df, 1), k in j+1:size(preference_df, 1)) +
    (fld(size(preference_df, 1), 3) + size(preference_df, 1)%3 - sum(z)) * RHO)

# Create constraints

# Each person can only be matched with two people
@constraint(m3, [i=1:size(preference_df, 1)], sum(z[i,j,k] for j in 1:size(preference_df, 1), k in 1:size(preference_df, 1)) <= 2)
@constraint(m3, [j=1:size(preference_df, 1)], sum(z[i,j,k] for i in 1:size(preference_df, 1), k in 1:size(preference_df, 1)) <= 2)
@constraint(m3, [k=1:size(preference_df, 1)], sum(z[i,j,k] for i in 1:size(preference_df, 1), j in 1:size(preference_df, 1)) <= 2)

#@constraint(m3, [i=1:size(preference_df, 1)], sum(z[i,k,k] for k in 1:size(preference_df, 1)) <= 1)
#@constraint(m3, [k=1:size(preference_df, 1)], sum(z[k,j,k] for j in 1:size(preference_df, 1)) == 0)
#@constraint(m3, [k=1:size(preference_df, 1)], sum(z[i,k,k] for i in 1:size(preference_df, 1)) == 0)

#@constraint(m1, [k=1:size(preference_df, 1)], sum(z[i,j,k] for i in 1:size(preference_df, 1) for j in 1:size(preference_df, 1)) <= 2)

#religion_df = Matrix(people_df[:, [:agnostic, :atheist, :buddhist, :catholic, :hindu, :jewish, :mormon, :muslim, :protestant, :unaffiliated]])

for i in 1:size(z, 1)
    for j in i:size(z, 1)
        for k in j:size(z, 3)

            # Make sure that if i is matched with j, then j is matched with i
            @constraint(m3, z[i,j, k] == z[j,i,k])
            @constraint(m3, z[i,j, k] == z[j,k,i])
            @constraint(m3, z[i,j, k] == z[k,j,i])
            @constraint(m3, z[i,j, k] == z[k,i,j])
            @constraint(m3, z[i,j, k] == z[i,k,j])

            # Different gender matching. >1 girl > 1 guy
            @constraint(m3, (people_df[i, :gender] + people_df[j, :gender] + people_df[k, :gender] - 2) * z[i,j,k] <=0 )
            @constraint(m3, (people_df[i, :gender] + people_df[j, :gender] + people_df[k, :gender] - 1) * z[i,j,k] >=0 )

            # People with strong religious preferences are matched with someone sharing the same religious preference
            # @constraint(m1, (1 - sum(religion_df[i, k] * religion_df[j, k] for k in 1:size(religion_df)[2])) * t[i,j] * z[i,j] <= 0)
            # @constraint(m1, t[i,j] >= people_df[i, :imprelig])
            # @constraint(m1, t[i,j] >= people_df[j, :imprelig])
        end
    end
end

optimize!(m3)

In [56]:
zij = value.(z)

100×100×100 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  